In [1]:
import torch
import copy
import numpy as np
import itertools
import random
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
import matplotlib
from tqdm import tqdm
matplotlib.use("TkAgg")
BOLD = "\033[1m"
RESET = "\033[0m"

In [3]:
class Environment():
    
    #fields:
    #a grid of size defined by parameter
    #b, a tuple repsenting the delivery location
    #a, a tuple representing the package location
    #size, an int representing the horizontal and vertical dimensions of the grid
    def __init__(self, size:int, verbose=0):
        self.size = size
        self.actions = ['north', 'south', 'west', 'east']
        self.reward_pickup = 500
        self.reward_deliver = 500
        self.reward_handover = 1000
        self.reward_move = -5
        self.reward_stuck = -100
        self.reset_env()
        self.verbose = verbose
    
    def reset_env(self):
        self.a=(0,0)
        self.b=(self.size-1,self.size-1)
        
        combinations = list(itertools.product(list(range(self.size)), repeat=2))
        # Remove (0, 0) and (size-1, size-1)
        combinations = [(x, y) for x, y in combinations if (x, y) != (0, 0) and (x, y) != (self.size-1, self.size-1)]
        # rd.shuffle(combinations)
        agentA1, agentA2, agentB1, agentB2 = random.sample(combinations, 4)
        self.agentsA_coord = [agentA1, agentA2]
        self.agentsB_coord = [agentB1, agentB2]

        self.agentsA_collect = [0,0]
        self.agentsB_collect = [0,0]

        self.terminated = False

        self.agentsA_step = [0,0]
        self.agentsB_step = [0,0]
        self.test_length = 0

        self.packages_delivered = 0

        self.setup_grid()

    # def coordinates_to_noisy_ohv(self, coordinate:int):
    #     ohv = np.zeros(4)
    #     noise = np.random.normal(-1e-5, 1e-5, ohv.shape)
    #     ohv[coordinate] = 1
    #     ohv = ohv + noise
    #     return ohv
    #     this is a legacy method from a one-hot representation of it
    
    # def get_optimal_distance(self):
    #     return abs(self.agent_coords[0]-self.a[0]) + abs(self.agent_coords[0]-self.a[1]) + abs(self.a[0]-self.b[0]) + abs(self.a[1]-self.b[1])


    #method to set up the original grid including a location
    # def setup_grid(self):
    #     grid = np.zeros((self.size, self.size))
    #     self.grid = grid
    #     grid[self.a] = 1
    #     grid[self.b] = 2
    #     for agentA in self.agentsA_coord:
    #         grid[agentA] = 3
    #     for agentB in self.agentsB_coord:
    #         grid[agentB] = 4

    def get_state(self):
        #This method previously unpacked the coordinates into a tuple which can be used to index the qmatrix or for easy incorporation to the Deep Q Network
        # coords = [self.agent_coords[0], self.agent_coords[1], self.a[0], self.a[1], self.b[0], self.b[1]]
        # coords = [self.coordinates_to_noisy_ohv(c) for c in coords]
        # coords.append(np.array(self.collected))
        # return np.hstack(coords)
        # return np.ndarray.flatten(np.array(coords))
        return np.array([self.agentsA_coord[0][0],self.agentsA_coord[0][1],self.agentsA_coord[1][0],self.agentsA_coord[1][1],
                         self.agentsB_coord[0][0],self.agentsB_coord[0][1],self.agentsB_coord[1][0],self.agentsB_coord[1][1],
                         self.agentsA_collect[0], self.agentsA_collect[1], self.agentsB_collect[0], self.agentsB_collect[1]])

    
    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, actionA, actionB):
        x, y = self.a
        self.grid[x, y] = 1  # Place "a" on the grid
        x, y = self.b
        self.grid[x, y] = 2  # Place "b" on the grid

        agentsA_reward = [0,0]
        agentsB_reward = [0,0]
        agentsA_deliver = [0,0]
        agentsB_deliver = [0,0]

        for i in range(len(self.agentsA_step)):
            self.agentsA_step[i] += 1
        for i in range(len(self.agentsB_step)):
            self.agentsB_step[i] += 1
        
        self.test_length += 1

        agentsA_coord_old = self.agentsA_coord
        agentsB_coord_old = self.agentsB_coord

        #Assign new coordinates for agent A to exist at
        for i in range(len(self.agentsA_coord)):
            new_x, new_y = self.agentsA_coord[i]
            if actionA[i] == 'north' and self.agentsA_coord[i][1] > 0:
                new_y -= 1
            elif actionA[i] == 'south' and self.agentsA_coord[i][1] < self.size - 1:
                new_y += 1
            elif actionA[i] == 'west' and self.agentsA_coord[i][0] > 0:
                new_x -= 1
            elif actionA[i] == 'east' and self.agentsA_coord[i][0] < self.size - 1:
                new_x += 1

            #Update the grid based on new agent coordinates
            self.grid[self.agentsA_coord[i]] = 0
            self.agentsA_coord[i] = (new_x, new_y)
            self.grid[self.agentsA_coord[i]] = 3

        #Assign new coordinates for agent B to exist at
        for i in range(len(self.agentsB_coord)):
            new_x, new_y = self.agentsB_coord[i]
            if actionB[i] == 'north' and self.agentsB_coord[i][1] > 0:
                new_y -= 1
            elif actionB[i] == 'south' and self.agentsB_coord[i][1] < self.size - 1:
                new_y += 1
            elif actionB[i] == 'west' and self.agentsB_coord[i][0] > 0:
                new_x -= 1
            elif actionB[i] == 'east' and self.agentsB_coord[i][0] < self.size - 1:
                new_x += 1

            #Update the grid based on new agent coordinates
            self.grid[self.agentsB_coord[i]] = 0
            self.agentsB_coord[i] = (new_x, new_y)
            self.grid[self.agentsB_coord[i]] = 4
        
        # Agent A:
        for i in range(len(self.agentsA_coord)):
            if self.agentsA_collect[i] == 0:
                if self.agentsA_coord[i] == self.a:
                    if self.verbose>2:
                        print("A{} pickup".format(i))
                    # self.agentsA_step[i] = 0
                    self.agentsA_collect[i]=1
                    agentsA_reward[i] = self.reward_pickup
                else:
                    agentsA_reward[i] = self.reward_move #-(self.agentsA_step[i]**2)
            else:
                for j in range(len(self.agentsB_coord)):
                    if self.agentsA_coord[i] == agentsB_coord_old[j]:
                        if self.agentsB_coord[j] == agentsA_coord_old[i]:
                            if self.agentsB_collect[j] == 0:
                                if self.agentsA_collect[i] == 1:
                                    self.agentsA_collect[i]=0
                                    agentsA_deliver[i]=1
                                    self.agentsB_collect[j]=1
                                    agentsA_reward[i] = self.reward_handover - ((np.abs(self.agentsA_coord[i][0] - self.a[0]) + np.abs(self.agentsA_coord[i][1] - self.a[1])) ** 2)
                                    agentsB_reward[j] = self.reward_handover - ((np.abs(self.agentsB_coord[j][0] - self.b[0]) + np.abs(self.agentsB_coord[j][1] - self.b[1])) ** 2)
                                    if self.verbose>2:
                                        print("A{} B{} handover".format(i,j))
                                        print("\tagentsA_reward",agentsA_reward[i], self.agentsA_step[i])
                                        print("\tagentsB_reward",agentsB_reward[j], self.agentsB_step[j])
                                    self.agentsA_step[i]=0 ; self.agentsB_step[j]=0

                if agentsA_deliver[i]==0:
                    agentsA_reward[i] = self.reward_move #-(self.agentsA_step[i]**2)

            # if self.agentsA_coord[i] == agentsA_coord_old[i]:
            #     agentsB_reward[i] = self.reward_stuck

        # Agent B:
        for i in range(len(self.agentsB_coord)):
            if self.agentsB_collect[i] == 1:
                if self.agentsB_coord[i] == self.b:
                    if self.verbose>2:
                        print("B{} deliver".format(i))
                    # self.agentsB_step[i]=0
                    agentsB_deliver[i]=1
                    self.agentsB_collect[i]=0
                    agentsB_reward[i] = self.reward_deliver
                    self.packages_delivered += 1
                else:
                    agentsB_reward[i] = self.reward_move #-(self.agentsB_step[i]**2)
            else:
                agentsB_reward[i] = self.reward_move #-(self.agentsB_step[i]**2)

            # if self.agentsB_coord[i] == agentsB_coord_old[i]:
            #     agentsB_reward[i] = self.reward_stuck

        if self.test_length > 200:
            if self.verbose>2:
                print("terminated")
            self.terminated = True

        if self.verbose>3:
            print("agentsA_reward",agentsA_reward)
            print("agentsB_reward",agentsB_reward)

        return agentsA_reward, agentsB_reward, agentsA_deliver, agentsB_deliver, self.packages_delivered, self.terminated

    def get_grid(self):
        return self.grid.tolist()

    #method to set up the original grid including a location
    def setup_grid(self):
        self.grid = np.zeros((self.size, self.size))
            
        x, y = self.a
        self.grid[x, y] = 1  # Place "a" on the grid

        x, y = self.b
        self.grid[x, y] = 2  # Place "b" on the grid

        # Place the "home" location (0) on the grid
        for i in range(len(self.agentsA_coord)):
            x, y = self.agentsA_coord[i]
            self.grid[x, y] = 3

        for i in range(len(self.agentsB_coord)):
            x, y = self.agentsB_coord[i]
            self.grid[x, y] = 4
    
    def plot_grid(self, snapshot, ax=None):
        if ax is None:
            fig, ax = plt.subplots()
            ax.set_facecolor('white')
        else:
            ax.clear()

        # Plot the grid
        ax.imshow(np.array([[0]]), cmap="bone", extent=[0, self.size, 0, self.size])

        for i in range(self.size):
            for j in range(self.size):
                cell_value = snapshot[i][j]
                images_to_display = []

                if cell_value == 3:
                    images_to_display.append(OffsetImage(agent_img, zoom=0.08, alpha=0.5))
                elif cell_value == 4:
                    images_to_display.append(OffsetImage(agent_img2, zoom=0.05, alpha=0.5))
                elif cell_value == 1:
                    images_to_display.append(OffsetImage(package_img, zoom=0.03, alpha=0.75))
                elif cell_value == 2:
                    images_to_display.append(OffsetImage(destinationB_img, zoom=0.05, alpha=0.75))

                for image in images_to_display:
                    ab = AnnotationBbox(image, (j + 0.5, self.size - i - 0.5), frameon=False)
                    ax.add_artist(ab)

        # Set axis properties
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_xticks(np.arange(self.size) + 1)
        ax.set_yticks(np.arange(self.size) + 1)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True, linewidth=2, color='white')

        # Set title
        ax.set_title("Package Delivery Agent")

        return ax
agent_img = plt.imread('agent.jpg')
agent_img2 = plt.imread("agent2.png")
package_img = plt.imread('package.jpg')
destinationB_img = plt.imread('destinationB.jpg')

In [4]:
class DQN:
    def __init__(self, learning_rate, state_size=100, action_size = 4):
        l1 = state_size
        l2 = 24
        l3 = 24
        l4 = action_size
        self.model = torch.nn.Sequential(
        torch.nn.Linear(l1, l2),
        torch.nn.ReLU(),
        torch.nn.Linear(l2, l3),
        torch.nn.ReLU(),
        torch.nn.Linear(l3,l4))

        self.model2 = copy.deepcopy(self.model)
        self.model2.load_state_dict(self.model.state_dict())
        self.loss_fn = torch.nn.MSELoss()
        self.learning_rate = learning_rate
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

# The function "update_target" copies the state of the prediction network to the target network. You need to use this in regular intervals.
    def update_target(self):
        self.model2.load_state_dict(self.model.state_dict())

# The function "get_qvals" returns a numpy list of qvals for the next_state given by the argument based on the prediction network.
    def get_qvals(self, next_state):
        state1 = torch.from_numpy(next_state).float()
        qvals_torch = self.model(state1)
        qvals = qvals_torch.data.numpy()
        return qvals

# The function "get_maxQ" returns the maximum q-value for the state given by the argument based on the target network.
    def get_maxQ(self,state):
        return torch.max(self.model2(torch.from_numpy(state).float())).float()

# The function "train_one_step_new" performs a single training step.
# It returns the current loss (only needed for debugging purposes).
# Its parameters are three parallel lists: a minibatch of states, a minibatch of actions,
# a minibatch of the corresponding TD targets and the discount factor.
    def train_one_step(self, states, actions, targets):
        targets_reply = []
        state1_batch = torch.from_numpy(states).float()
        # state1_batch = torch.cat([torch.from_numpy(s).float() for s in states])
        action_batch = torch.Tensor(actions)
        Q1 = self.model(state1_batch)
        X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
        Y = torch.tensor(targets).float()
        loss = self.loss_fn(X, Y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [5]:
class AgentA(DQN):
    def __init__(self, statespace_size=12, gamma=0.99, learning_rate=0.997, start_epsilon=1.0,
                 epsilon_decay_factor=0.999, min_epsilon=0.05, replay_buffer_size=10000,
                 batch_size=200, network_copy_frequency=10):
        super().__init__(learning_rate=learning_rate, state_size=statespace_size, action_size=4)  # Call the parent constructor
        self.statespace_size = statespace_size
        self.gamma = gamma
        # self.learning_rate = learning_rate
        self.memory = []  # Using a list instead of a deque
        self.epsilon = start_epsilon
        self.epsilon_min = min_epsilon
        self.epsilon_decay_factor = epsilon_decay_factor
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        self.network_copy_frequency = network_copy_frequency
        self.steps_since_copy = 0  # Counter for network copy
    
    def remember(self, state, actionA, agentsA_reward, agentsA_deliver, next_state):
        self.memory.append((state, actionA, agentsA_reward, agentsA_deliver, next_state))
        while len(self.memory) > self.replay_buffer_size:
            self.memory.pop(0)  # Remove the oldest experience

    def update_target(self):
        if self.steps_since_copy % self.network_copy_frequency == 0:
            super().update_target()

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(range(4))  # Random action
        q_values = self.get_qvals(state)
        return np.argmax(q_values)  # Greedy action

    def process_minibatch(self, minibatch):
        states = []
        actions = []
        targets = []
        for state, actionA, agentsA_reward, agentsA_deliver, next_state in minibatch:
            q_values = self.get_qvals(state)
                
            if agentsA_deliver==1:
                q_values[actionA] = agentsA_reward
            else:
                q_values[actionA] = agentsA_reward + self.gamma * self.get_maxQ(next_state)
            
            actions.append(actionA)
            targets.append(q_values[actionA])
        
            states.append(state)
            
        return np.array(states), np.array(actions), np.array(targets)

    def save_model_parameters(self, destination):
        torch.save(self.model2.state_dict(), destination)

    def load_model_parameters(self, path):
        self.model2.load_state_dict(torch.load(path))

In [6]:
class AgentB(DQN):
    def __init__(self, statespace_size=12, gamma=0.99, learning_rate=0.997, start_epsilon=1.0,
                 epsilon_decay_factor=0.999, min_epsilon=0.05, replay_buffer_size=10000,
                 batch_size=200, network_copy_frequency=10):
        super().__init__(learning_rate=learning_rate, state_size=statespace_size, action_size=4)  # Call the parent constructor
        self.statespace_size = statespace_size
        self.gamma = gamma
        self.memory = []  # Using a list instead of a deque
        self.epsilon = start_epsilon
        self.epsilon_min = min_epsilon
        self.epsilon_decay_factor = epsilon_decay_factor
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        self.network_copy_frequency = network_copy_frequency
        self.steps_since_copy = 0  # Counter for network copy

    def remember(self, state, actionB, agentsB_reward, agentsB_deliver, next_state):
        self.memory.append((state, actionB, agentsB_reward, agentsB_deliver, next_state))
        while len(self.memory) > self.replay_buffer_size:
            self.memory.pop(0)  # Remove the oldest experience

    def update_target(self):
        if self.steps_since_copy % self.network_copy_frequency == 0:
            super().update_target()

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(range(4))  # Random action
        q_values = self.get_qvals(state)
        return np.argmax(q_values)  # Greedy action

    def process_minibatch(self, minibatch):
        states = []
        actions = []
        targets = []
        for state, actionB, agentsB_reward, agentsB_deliver, next_state in minibatch:
            q_values = self.get_qvals(state)

            if agentsB_deliver==1:
                q_values[actionB] = agentsB_reward
            else:
                q_values[actionB] = agentsB_reward + self.gamma * self.get_maxQ(next_state)
            
            actions.append(actionB)
            targets.append(q_values[actionB])
        
            states.append(state)
            
        return np.array(states), np.array(actions), np.array(targets)

    def save_model_parameters(self, destination):
        torch.save(self.model2.state_dict(), destination)

    def load_model_parameters(self, path):
        self.model2.load_state_dict(torch.load(path))

In [7]:
def create_one_hot_vector(my_x, my_y, friend_x, friend_y, other1_x, other1_y, other2_x, other2_y, my_flag, friend_flag, other1_flag, other2_flag, grid_size=5):
    
    # Initialize a grid to represent the positions and flags of agents
    grid = np.zeros((grid_size, grid_size, 2, 4))

    # Set the positions of agents from group A
    if my_flag == 0:
        grid[my_x, my_y, 0, 0] = 1
    else:
        grid[my_x, my_y, 1, 0] = 1

    if friend_flag == 0:
        grid[friend_x, friend_y, 0, 1] = 1
    else:
        grid[friend_x, friend_y, 1, 1] = 1

    # Set the positions of agents from group B
    if other1_flag == 0:
        grid[other1_x, other1_y, 0, 2] = 1
    else:
        grid[other1_x, other1_y, 1, 2] = 1

    if other2_flag == 0:
        grid[other2_x, other2_y, 0, 3] = 1
    else:
        grid[other2_x, other2_y, 1, 3] = 1


    # Add a small noise to all the zeros
    noise_level = 1e-6
    grid += np.random.uniform(0, noise_level, size=grid.shape)

    # Flatten the grid into a one-hot vector
    one_hot_vector = grid.reshape(-1)

    return one_hot_vector

In [8]:
def run_visualisation(agentA, agentB, max_steps=50, size=5):
    #Assumes a fully trained transport agent. 
    #Target network should either have been trained in the preceeding step, or loaded from a pickled pytorch weights file
    env = Environment(size=size,verbose=3)
    # Create a figure and axis outside the loop
    fig, ax = plt.subplots()
    ax.set_facecolor('white')

    # Initialize the plot once with the initial grid
    m = ax.imshow(env.grid, cmap="bone", extent=[0, env.size, 0, env.size])
    snapshots = []
    for step in range(max_steps):
        state = env.get_state()
        stateA1 = np.array([state[0], state[1], state[2], state[3], state[4], state[5], state[6], state[7], state[8], state[9], state[10], state[11]])
        stateA2 = np.array([state[2], state[3], state[0], state[1], state[4], state[5], state[6], state[7], state[9], state[8], state[10], state[11]])
        stateB1 = np.array([state[4],state[5], state[6],state[7], state[0], state[1],state[2],state[3],state[10], state[11], state[8],state[9]])
        stateB2 = np.array([state[6],state[7], state[4],state[5], state[0], state[1],state[2],state[3], state[11], state[10], state[8],state[9]])

        actionA1= np.argmax(agentA.get_qvals(stateA1))
        actionA2 = np.argmax(agentA.get_qvals(stateA2))
        actionB1= np.argmax(agentB.get_qvals(stateB1))
        actionB2 = np.argmax(agentB.get_qvals(stateB2))

        actionA = [env.actions[actionA1], env.actions[actionA2]]
        actionB = [env.actions[actionB1], env.actions[actionB2]]

        _, _, _, _, packages_delivered, terminated  = env.move_agent(actionA, actionB)
        snapshots.append(env.get_grid())

        if terminated or packages_delivered>5:
            break
    print(BOLD, "total packages_delivered", packages_delivered, RESET)
    for s in snapshots:
        env.plot_grid(s, ax)
        # Redraw the plot
        fig.canvas.draw()
        plt.pause(0.2)  # Add a short pause for visualization
    # plt.pause(1.5)
    plt.close()
    return None

In [9]:
def train_function(env, agentA, agentB, episodes=100):
    loss_history = []
    empirical_reward_history = []
    # distance_history =[]

    for episode in (range(episodes)): #tqdm
        print(BOLD, "episode", episode, RESET)
        env.reset_env()
        state = env.get_state()  # Reset the environment and get the initial state
        
        # steps_to_opt = env.get_optimal_distance()
        steps_in_episode = 0
        # done = False
        terminated = False
        total_reward = 0
        packages_delivered = 0
        episode_losses = []

        if (episode>0 and episode%25 == 0) or (episode == episodes-1):
            agentA.save_model_parameters("logs/target_params_A_{}_onehot.pt".format(episode))
            agentB.save_model_parameters("logs/target_params_B_{}_onehot.pt".format(episode))

        # print("epsilon", agentA.epsilon)
        
        while (packages_delivered<5 and not terminated):

            stateA1 = np.array([state[0], state[1], state[2], state[3], state[4], state[5], state[6], state[7], state[8], state[9], state[10], state[11]])
            stateA2 = np.array([state[2], state[3], state[0], state[1], state[4], state[5], state[6], state[7], state[9], state[8], state[10], state[11]])
            stateB1 = np.array([state[4],state[5], state[6],state[7], state[0], state[1],state[2],state[3],state[10], state[11], state[8],state[9]])
            stateB2 = np.array([state[6],state[7], state[4],state[5], state[0], state[1],state[2],state[3], state[11], state[10], state[8],state[9]])

            # print("state", state)
            actionA1 = agentA.act(stateA1)
            actionA2 = agentA.act(stateA2)

            actionB1 = agentB.act(stateB1)
            actionB2 = agentB.act(stateB2)

            actionA = [env.actions[actionA1], env.actions[actionA2]]
            actionB = [env.actions[actionB1], env.actions[actionB2]]

            agentsA_reward, agentsB_reward, agentsA_deliver, agentsB_deliver, packages_delivered, terminated = env.move_agent(actionA, actionB)
            
            next_state = env.get_state()
            next_stateA1 = np.array([next_state[0], next_state[1], next_state[2], next_state[3], next_state[4], next_state[5], next_state[6], next_state[7], next_state[8], next_state[9], next_state[10], next_state[11]])
            next_stateA2 = np.array([next_state[2], next_state[3], next_state[0], next_state[1], next_state[4], next_state[5], next_state[6], next_state[7], next_state[9], next_state[8], next_state[10], next_state[11]])
            next_stateB1 = np.array([next_state[4],next_state[5], next_state[6],next_state[7], next_state[0], next_state[1],next_state[2],next_state[3],next_state[10], next_state[11], next_state[8],next_state[9]])
            next_stateB2 = np.array([next_state[6],next_state[7], next_state[4],next_state[5], next_state[0], next_state[1],next_state[2],next_state[3], next_state[11], next_state[10], next_state[8],next_state[9]])

            # next_state, reward, done = env.step(env.actions[action])
            # total_reward += reward
            agentA.remember(stateA1, actionA1, agentsA_reward[0], agentsA_deliver[0], next_stateA1)
            agentA.remember(stateA2, actionA2, agentsA_reward[1], agentsA_deliver[1], next_stateA2)
            agentB.remember(stateB1, actionB1, agentsB_reward[0], agentsB_deliver[0], next_stateB1)
            agentB.remember(stateB2, actionB2, agentsB_reward[1], agentsB_deliver[1], next_stateB2)

            state = next_state
            steps_in_episode += 1 

            # Training A
            if len(agentA.memory) > agentA.batch_size:
                minibatch_indices = np.random.choice(len(agentA.memory), agentA.batch_size, replace=False)
                minibatch = [agentA.memory[i] for i in minibatch_indices]
                states_batch, actions_batch, targets_batch = agentA.process_minibatch(minibatch)
                loss = agentA.train_one_step(states_batch, actions_batch, targets_batch)
                episode_losses.append(loss)


            # Training B
            if len(agentB.memory) > agentB.batch_size:
                minibatch_indices = np.random.choice(len(agentB.memory), agentB.batch_size, replace=False)
                minibatch = [agentB.memory[i] for i in minibatch_indices]
                states_batch, actions_batch, targets_batch = agentB.process_minibatch(minibatch)
                loss = agentB.train_one_step(states_batch, actions_batch, targets_batch)
                episode_losses.append(loss)
        
        #Cleanup step

        agentA.epsilon = max(agentA.epsilon * agentA.epsilon_decay_factor, agentA.epsilon_min)
        agentB.epsilon = max(agentB.epsilon * agentB.epsilon_decay_factor, agentB.epsilon_min)

        if len(episode_losses) > 0:
            mean_loss = sum(episode_losses)/len(episode_losses)
        else:
            mean_loss = "n/a"

        agentA.steps_since_copy += 1
        agentA.update_target()
        agentB.steps_since_copy += 1
        agentB.update_target()

        # print("state", state)
        # print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon} | Loss: {mean_loss}")
        # print(f"Collected: {env.collected} | Terminated: {terminated} | Done: {done}")
        
        # distance_history.append(steps_in_episode - steps_to_opt + 1)
        loss_history.append(mean_loss)
        empirical_reward_history.append(total_reward)
    return loss_history, empirical_reward_history #, distance_history

In [10]:
# if __name__ == "__main__":
env = Environment(5, verbose=3) # Create the environment

gamma = 0.8; learning_rate = 0.01; epsilon_decay_factor = 0.97; min_epsilon=0.05; replay_buffer_size=10000; batch_size=300; network_copy_frequency=15
episodes = 400

agentA = AgentA(gamma=gamma, learning_rate=learning_rate, epsilon_decay_factor=epsilon_decay_factor,
                 min_epsilon=min_epsilon, replay_buffer_size=replay_buffer_size,
                 batch_size=batch_size, network_copy_frequency=network_copy_frequency)
agentB = AgentB(gamma=gamma, learning_rate=learning_rate, epsilon_decay_factor=epsilon_decay_factor,
                 min_epsilon=min_epsilon, replay_buffer_size=replay_buffer_size,
                 batch_size=batch_size, network_copy_frequency=network_copy_frequency)

losses, rewards = train_function(env, agentA, agentB, episodes=episodes) #, distance_history
np.save("losses.npy", losses, allow_pickle=False)
np.save("rewards.npy", rewards, allow_pickle=False)
# np.save("distances.npy", distance_history, allow_pickle=False)

 episode 0 
A0 pickup
A0 B1 handover
	agentsA_reward 1000 12
	agentsB_reward 936 12
A0 pickup
A0 B0 handover
	agentsA_reward 996 13
	agentsB_reward 964 25
B1 deliver
A0 pickup
B0 deliver
A0 B1 handover
	agentsA_reward 984 55
	agentsB_reward 984 68
A1 pickup
A1 B0 handover
	agentsA_reward 999 91
	agentsB_reward 951 66
A0 pickup
B0 deliver
B1 deliver
A0 B1 handover
	agentsA_reward 964 70
	agentsB_reward 996 70
terminated
 episode 1 
A0 pickup
A0 B0 handover
	agentsA_reward 999 33
	agentsB_reward 951 33
A0 pickup
A0 B1 handover
	agentsA_reward 999 6
	agentsB_reward 951 39
A0 pickup
B1 deliver
A0 B1 handover
	agentsA_reward 964 43
	agentsB_reward 996 43
A0 pickup
B0 deliver
A0 B0 handover
	agentsA_reward 1000 90
	agentsB_reward 936 139
A0 pickup
B0 deliver
terminated
 episode 2 
A1 pickup
A1 B0 handover
	agentsA_reward 975 54
	agentsB_reward 991 54
B0 deliver
A1 pickup
A1 B0 handover
	agentsA_reward 991 79
	agentsB_reward 975 79
A0 pickup
A0 B1 handover
	agentsA_reward 999 201
	agentsB_rew

In [14]:
# check visualization
# agentA.load_model_parameters(path = 'logs/target_params_A_200_onehot.pt')
# agentB.load_model_parameters(path = 'logs/target_params_B_200_onehot.pt')
run_visualisation(agentA, agentB)

A0 pickup
A1 pickup
A0 B0 handover
	agentsA_reward 975 8
	agentsB_reward 991 8
B0 deliver
A1 B0 handover
	agentsA_reward 936 12
	agentsB_reward 1000 4
B0 deliver
A0 pickup
A0 B0 handover
	agentsA_reward 984 9
	agentsB_reward 984 5
A1 pickup
A0 pickup
B0 deliver
A1 B1 handover
	agentsA_reward 984 12
	agentsB_reward 984 24
A0 B0 handover
	agentsA_reward 975 9
	agentsB_reward 991 9
A1 pickup
B1 deliver
B0 deliver
A0 pickup
A1 B0 handover
	agentsA_reward 975 9
	agentsB_reward 991 7
A0 B1 handover
	agentsA_reward 975 10
	agentsB_reward 991 12
B0 deliver
 total packages_delivered 6 


In [12]:
round_losses = [round(loss, 2) for loss in losses if loss != "n/a"]
plt.plot(round_losses, label='Loss')
plt.xlabel('Epochs (or training steps)')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()